This can be used to initalize the database for many of the different experiments below. 

In [11]:
from Juliet_DB_loader import TestCase, Folder, SubFolder, sessionmaker, engine
from sqlalchemy import func
from sqlalchemy.orm import sessionmaker 
import re

Session = sessionmaker(bind=engine)
session = Session()


The below code can be used to verify a specific test cases's content and line number before any modifications are made


In [ ]:
test_cases = session.query(TestCase).filter_by(id=47105).first()
line_content = dict()
lines = test_cases.file_content.split('\n')
multiline_comment = False
for line_num, line in enumerate(lines, start=1):
    line_content[line_num] = line

# Print the dictionary vertically
for line_num, line in line_content.items():
    print(f"{line_num}: {line}")

This block can be used to check the vulnerable line of a specific test file. 

In [34]:
test_case = session.query(TestCase).filter_by(id=10000).first()
line_content = dict()
lines = test_case.file_content.split('\n')
vulnerable_line = int(test_case.vulnerability_location)

print(f"Vulnerable line: {vulnerable_line}")
print(f"Total lines in file: {len(lines)}")

for line_num, line in enumerate(lines, start=1):
    line_content[line_num] = line    

# Check if vulnerable_line is a valid key in line_content
if vulnerable_line in line_content:
    print(f"{vulnerable_line}: {line_content[vulnerable_line]}")
else:
    print(f"Vulnerable line {vulnerable_line} is not in the file content.")

Vulnerable line: 43
Total lines in file: 133
43:         strcpy(data, source);


2-6-24 


This block's goal is to get a batch of files and put them into a new folder for experimenting my comment_cleaner.

2-7 
I have sucessfully done this, and I am going to take a batch of programs from each langauge to make my first attempt at a corpus for tokenization. 

Corpus Version 1.0
Important Notes: I am using the suboptimal comment stripper so there are a fair amount of comments still in this corpus. 


In [10]:
from Juliet2 import Cases,  sessionmaker, engine2
from sqlalchemy import and_, func
from sqlalchemy.orm import relationship, sessionmaker
import rcmpy
import tqdm
Session1 = sessionmaker(bind=engine2)
session = Session1()

# This function will need to be more developed for getting everything into the training, testing, and validation sets. 
# Function to get 10 random test cases for a given file extension
def get_corpus(session):
    # Move the file initialization outside the loop
    with open("Juliet_corpus2.0.txt", "w") as write_file:
        Files = session.query(Cases.file_content).all()
        for file in Files:
            # Access the first item of the tuple to get the file content
            write_file.write(file[0])

# Call the function with the desired parameters
get_corpus(session)




KeyboardInterrupt: 

Testing Set for new comment remover.

In [1]:
from Juliet_DB_loader import TestCase, Folder, SubFolder, sessionmaker, engine
from sqlalchemy import and_, func
from sqlalchemy.orm import relationship, sessionmaker
from sqlalchemy.exc import SQLAlchemyError
from comment_cleaner import strip
from comment_cleaner import remove_empty_lines
import random, os
Session = sessionmaker(bind=engine)
session = Session()

def get_test_cases(session, file_extension, num_cases):
    query = session.query(TestCase)

    test_cases = query.filter(TestCase.file_extension == file_extension).limit(num_cases).all()

    test_case_ids = [test_case.id for test_case in test_cases]
    return test_cases, test_case_ids

c_test_cases, c_ids = get_test_cases(session, '.c', 3000)
cpp_test_cases, cpp_ids = get_test_cases(session, '.cpp', 3000)
java_test_cases, java_ids = get_test_cases(session, '.java', 3000)

all_cases = c_test_cases + cpp_test_cases + java_test_cases
corpus_file_path = 'test_cases_RCM'

os.makedirs('test_cases_RCM', exist_ok=True)
for i, case in enumerate(all_cases):
    # Process the file content
    lines = case.file_content.split('\n')
    case.file_content = '\n'.join(lines)

    # Write the processed content to a new file
    with open(f'test_cases_RCM/test_case_{i}.txt', 'w') as f:
        f.write(case.file_content)

    # Append the processed content to the corpus file
    with open(corpus_file_path, 'a', encoding='utf-8') as corpus_file:
        corpus_file.write(case.file_content + '\n')


Generating LALR tables


1: 
2: 
3: 
4: 
5: 

6: 
7: 
8: 
9: 
10: 
11: 
12: 
13: 
14: 
15: 
16: 

17: 

18: package testcases.CWE400_Resource_Exhaustion.s01;
19: import testcasesupport.*;
20: 

21: import javax.servlet.http.*;
22: 

23: import java.io.BufferedReader;
24: import java.io.InputStreamReader;
25: import java.io.IOException;
26: import java.net.Socket;
27: 

28: import java.util.logging.Level;
29: 

30: import java.io.File;
31: import java.io.FileOutputStream;
32: import java.io.OutputStreamWriter;
33: import java.io.BufferedWriter;
34: 

35: 

36: public class CWE400_Resource_Exhaustion__connect_tcp_write_45 extends AbstractTestCase
37: {
38:     private int countBad;
39:     private int countGoodG2B;
40:     private int countGoodB2G;
41: 

42:     private void badSink() throws Throwable
43:     {
44:         int count = countBad;
45: 

46:         File file = new File("badSink.txt");
47:         FileOutputStream streamFileOutput = new FileOutputStream(file);
48:         OutputStreamWriter writerOu

PermissionError: [Errno 13] Permission denied: 'test_cases_RCM'